In [24]:
!pip install nltk
import nltk
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords  
from nltk.corpus import words
from collections import Counter 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Gensim
!pip install -U gensim==3.8.3
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# installing mallet
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# !unzip mallet-2.0.8.zip


import pandas as pd
import numpy as np
import datetime as dt

# Plotting tools
import matplotlib
from matplotlib import pylab
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Center outputs
from IPython.core.display import HTML, Markdown
HTML("""
<style>
.output_png, .output {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

import warnings
warnings.filterwarnings('once')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
df=pd.read_csv("../data/data_news.csv")

In [26]:
df=df.drop(columns=['Unnamed: 0','titulo'])
df.head()

,categoria,href,fecha,title
0,Economía,/economia/david-stern-el-hombre-que-transformo...,2020-01-01,"David Stern, the man who transformed the NBA i..."
1,Internacional,/mundo/internacional/como-un-pais-con-vinos-de...,2020-01-01,How a country with unknown wines conquered New...
2,Empresas,/economia/empresas/hombre-mas-rico-de-asia-ret...,2020-01-01,Asia's richest man challenges Amazon with new ...
3,Empresas,/economia/empresas/italia-inicia-el-camino-par...,2020-01-01,Italy begins the path to suspend concessions t...
4,Política,/peru/politica/elecciones-2020-jne-confirma-do...,2020-01-01,Elections 2020: JNE confirms two candidacies a...


In [22]:
headlines = []
for fecha in df['fecha'].unique():
    headlines.append(' '.join(str(x) for x in df[df['fecha'] ==fecha ].iloc[0:16,3]))
    
    

In [29]:
pd.DataFrame(list(zip(df['fecha'].unique(), headlines)))

,0,1
0,2020-01-01,"David Stern, the man who transformed the NBA i..."
1,2020-01-02,Keiko Fujimori: judge rejects psychiatric expe...
2,2020-01-03,JNJ: Prosecutor of the Nation voted against th...
3,2020-01-04,Elections 2020: citizens will be able to ask q...
4,2020-01-05,Graña y Montero knew of US$3 million delivered...
...,...,...
726,2021-12-27,Mesías Guevara: Castillo has made several mist...
727,2021-12-28,Rosendo Serna Román was sworn in as the new Mi...
728,2021-12-29,Elections 2022: JNE Single Window allows parti...
729,2021-12-30,Popular Force asks to summon Minister Avelino ...


In [30]:
label = pd.read_excel("../data/~$MONTO DE INTERVENCION.xlsx")

PermissionError: [Errno 13] Permission denied: '../data/~$MONTO DE INTERVENCION.xlsx'

In [23]:
headlines

["David Stern, the man who transformed the NBA into a multinational How a country with unknown wines conquered New York Asia's richest man challenges Amazon with new online portal Italy begins the path to suspend concessions to the Benetton family Elections 2020: JNE confirms two candidacies and declares another six unfounded Airbus exceeds its goal with 863 aircraft deliveries and displaces Boeing from first place Spouses have the right to block the withdrawal of CTS from workers Chinese central bank eases bank reserve policy to boost growth Senasa will be able to close businesses due to irregularities in the sale of seeds and fertilizers The price of the dollar fell in 2019, what factors motivated this result? Keiko Fujimori case: Giulliana Loza's tweets that Domingo Pérez questioned at the court hearing Produce: new Fondemi trust will allow granting loans of up to S/ 100,000 to mypes The WTO turns 25 wrapped in a crisis of reserved prognosis Marianella Ledesma will assume the presid

In [ ]:
' '.join(str(x) for x in df.iloc[1,0:25])

In [19]:
df[df['fecha'] =='2020-01-01' ].iloc[0:16,3]

0     David Stern, the man who transformed the NBA i...
1     How a country with unknown wines conquered New...
2     Asia's richest man challenges Amazon with new ...
3     Italy begins the path to suspend concessions t...
4     Elections 2020: JNE confirms two candidacies a...
5     Airbus exceeds its goal with 863 aircraft deli...
6     Spouses have the right to block the withdrawal...
7     Chinese central bank eases bank reserve policy...
8     Senasa will be able to close businesses due to...
9     The price of the dollar fell in 2019, what fac...
10    Keiko Fujimori case: Giulliana Loza's tweets t...
11    Produce: new Fondemi trust will allow granting...
12    The WTO turns 25 wrapped in a crisis of reserv...
13    Marianella Ledesma will assume the presidency ...
14    Elections 2020: JNE decides to accept the cand...
15    Israeli and German researchers predict El Niño...
Name: title, dtype: object

In [18]:
' '.join(str(x) for x in df[df['fecha'] =='2020-01-01' ].iloc[0:16,3])

"David Stern, the man who transformed the NBA into a multinational How a country with unknown wines conquered New York Asia's richest man challenges Amazon with new online portal Italy begins the path to suspend concessions to the Benetton family Elections 2020: JNE confirms two candidacies and declares another six unfounded Airbus exceeds its goal with 863 aircraft deliveries and displaces Boeing from first place Spouses have the right to block the withdrawal of CTS from workers Chinese central bank eases bank reserve policy to boost growth Senasa will be able to close businesses due to irregularities in the sale of seeds and fertilizers The price of the dollar fell in 2019, what factors motivated this result? Keiko Fujimori case: Giulliana Loza's tweets that Domingo Pérez questioned at the court hearing Produce: new Fondemi trust will allow granting loans of up to S/ 100,000 to mypes The WTO turns 25 wrapped in a crisis of reserved prognosis Marianella Ledesma will assume the preside